In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [5]:
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
llm=OpenAI(temperature=0.6)


In [6]:
#First chain generates hypothesis from field and specific problem

#first chain: take in a problem, generate 5 hypotheses
llm = OpenAI(temperature=0.5)
template= """
You are a hyper-intelligent scientist. Given the following {problem}, generate a numbered list containing five hypotheses for possible research directions. 
"""
prompt_template=PromptTemplate(input_variables=["problem"],template=template)
hypothesis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="hyplist")

In [7]:
#chain to choose from the best hypothesis
llm=OpenAI(temperature=0.5)
template2 = """
Choose the simplest hypothesis from this list: {hyplist}
"""
prompt_template2 = PromptTemplate(
    input_variables=["hyplist"], template=template2)
choice_chain = LLMChain(
    llm=llm, prompt=prompt_template2, output_key="choice")


In [8]:
# This is an LLMChain to create an experiment given a hypothesis.
llm = OpenAI(temperature=.7)
template = """You are a scientist. Given this hypothesis, it is your job to create a concrete, testable experiment to test the hypothesis. Give a numbered list of steps I should follow to do the experiment in my lab, being as specific as possible.

Hypothesis: {choice}
Scientist: Here's an experiment:"""
prompt_template = PromptTemplate(
    input_variables=["choice"], template=template)
experiment_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="experiment")


In [9]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
full_chain = SequentialChain(
    chains=[hypothesis_chain, choice_chain,experiment_chain],
    input_variables=["problem"],
    # Here we return multiple variables
    output_variables=["hyplist","choice", "experiment"],
    verbose=True)


In [10]:
#TODO: this will become HTML input routed through flask app
problem = input("Enter a problem")
output = full_chain({"problem":problem})



> Entering new SequentialChain chain...
Chain 0:
{'hyplist': '\n1. Developing more efficient irrigation systems to reduce water waste and increase crop yield \n2. Exploring the effects of different types of fertilizers on crop yield \n3. Examining the impact of climate change on crop yield \n4. Analyzing the effect of different farming techniques on crop yield \n5. Investigating the influence of various soil types on crop yield'}

Chain 1:
{'choice': '\nExploring the effects of different types of fertilizers on crop yield'}

Chain 2:
{'experiment': '\n\n1. Obtain three different types of fertilizers from a garden store or agricultural supply store.\n\n2. Select a plot of land that is suitable for crop growth and mark it off with a measuring tape and stakes.\n\n3. Plant the same type of crop in each section of the plot, making sure to evenly distribute the seeds.\n\n4. Water the plot with the same amount of water, using a sprinkler or hose.\n\n5. Apply the different types of fertilize